In [1]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os.path
import json
import pickle
import requests

def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 
            #   'https://www.googleapis.com/auth/script.projects', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file'
              ]
    creds_filename = 'test-daebong-service-account.json'  # 서비스 계정 파일 경로를 지정합니다.

    # 서비스 계정 파일을 사용하여 인증 정보를 로드합니다.
    creds = service_account.Credentials.from_service_account_file(creds_filename, scopes=SCOPES)
    return creds

def oauthByWeb():
    # 필요한 스코프를 지정합니다.
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/script.projects',
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
    ]
    creds = None
    # 'token.json' 파일이 존재하면, 저장된 인증 정보를 불러옵니다.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)

    # 저장된 인증 정보가 없거나, 유효하지 않은 경우 새로운 인증을 진행합니다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds_filename = 'oauth-new-daebong.json'
            flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
        # 새로운 인증 정보를 'token.json'에 저장합니다.
        with open('token.json', 'wb') as token:
            pickle.dump(creds, token)

    return creds



In [2]:
def apply_formula_to_muti_cells(service, spreadsheet_id, range_name, phone_number):
    formulas = []
    for i in range(11, 1011):  # 11행부터 1010행까지
        formula = f'=IF(LEN(A{i})=0,"", \"{phone_number}\")'
        formulas.append([formula])

    value_range_body = {
        'values': formulas
    }

    request = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=range_name, valueInputOption='USER_ENTERED', body=value_range_body)
    request.execute()


In [3]:
def get_values_from_range(service, spreadsheet_id, range_name):
    # Google Sheets API를 사용하여 특정 범위의 값 가져오기
    result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    # 값이 있는지 확인 후 반환
    values = result.get('values', [])

    return values

In [4]:
user_sheet_id = "1Ajz2g-pChLSoies5umhPMcgTp5m2MIjhljJOviPlWCU"
# spreadsheet_copy_id = "1p0WM4X2JztS1LfGKVdUXd3GTvBJJGYPNO0ya8ihfns4"
import time


if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    service_drive = build("drive", "v3", credentials=creds) 
    range_name = "셀러컨택포인트!A426:T437"
    # range_name = "셀러컨택포인트!A2:T4"
    # range_name = "셀러컨택포인트!A265:T267"
    user_data_list = get_values_from_range(service_sheets, user_sheet_id, range_name)
    print(user_data_list)

     
    for user_data in user_data_list:
        time.sleep(2)

        if user_data[10]:
            print("user_data", user_data[1])
            user_id = user_data[10]
            phone_number = "01051319792"
            if user_data[9]:
                phone_number = user_data[9]
            write_range = "발주서!I11:I"
            apply_formula_to_muti_cells(service_sheets, user_id, write_range, phone_number)
        






[['', '444', '이승민', '', '', '', '', '', '', '1090236254', '1cGSDZu2qK_Cx6jXa_qzlnX1fumXlyIUi2LIKVDX70Sw', 'https://docs.google.com/spreadsheets/d/1cGSDZu2qK_Cx6jXa_qzlnX1fumXlyIUi2LIKVDX70Sw', 'coffeesion28@gmail.com', '', '', '', '', '', '', '1'], ['', '445', '정영식', '', '', '', '', '', '', '1049345631', '1W52kNbCaDXafLTYxMfJ3pnMmzU8o1UIkYonsrmm2V1k', 'https://docs.google.com/spreadsheets/d/1W52kNbCaDXafLTYxMfJ3pnMmzU8o1UIkYonsrmm2V1k', 'shantklove486@gmail.com', '브로컴퍼니', '', '', '', '', '', '1'], ['', '446', '박서희', '', '', '', '', '', '', '1055393731', '1FAInvjJCqdCk1ZyoFWv-kUhF6pBmZpN43AK_z_Exkjs', 'https://docs.google.com/spreadsheets/d/1FAInvjJCqdCk1ZyoFWv-kUhF6pBmZpN43AK_z_Exkjs', 'kkomil620@gmail.com', '', '', '', '', '', '', '1'], ['', '447', '김서온', '', '', '', '', '', '', '1043757715', '1CwPGQ9T353ee1YZRy3HVB1KnF2ay0cZkA5LLUXpXrrU', 'https://docs.google.com/spreadsheets/d/1CwPGQ9T353ee1YZRy3HVB1KnF2ay0cZkA5LLUXpXrrU', 'ghkchd0705@gmail.com', '', '', '', '', '', '', '1'], ['', '

In [4]:
def apply_formula_to_muti_cell_ready(service, spreadsheet_id, range_name):
    formulas = []
    for i in range(11, 1011):  # 11행부터 1010행까지
        formula = f'=IF(AND(LEN(A{i})>0, LEN(B{i})>0, LEN(C{i})>0, LEN(D{i})>0, LEN(E{i})>0), \"발주완료\", "")'
        formulas.append([formula])

    value_range_body = {
        'values': formulas
    }

    request = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=range_name, valueInputOption='USER_ENTERED', body=value_range_body)
    request.execute()


In [5]:
# 특정 셀에 수식을 적용하는 함수
def apply_formula_to_cell_sheet(service, spreadsheet_id, cell_range):
    formula = f'=HYPERLINK("https://pf.kakao.com/_xdkxbCG", "입금자명을 수정하고 싶다면, 고객센터에 알려주세요(클릭)")'  # A2 셀에 적용할 수식
    value_range_body = {
        'values': [[formula]]  # 적용할 수식
    }
    
    request = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=cell_range,
        valueInputOption='USER_ENTERED',
        body=value_range_body
    )
    request.execute()

In [29]:
# # 특정 셀에 수식을 적용하는 함수
# def apply_formula_to_cell_sheet_for_cs(service, spreadsheet_id, cell_range):
#     formula = f'=HYPERLINK("https://daebongfarmers.notion.site/CS-3f5809dfece649d9986096fce0a1405f", "CS 접수하기(클릭)")'  # A2 셀에 적용할 수식
#     # formula = '=HYPERLINK("https://daebongfarmers.notion.site/CS-6717dad944344584b4aa870c8eb376a7", "발주서 작성 가이드")'
#     value_range_body = {
#         'values': [[formula]]  # 적용할 수식
#     }
    
#     request = service.spreadsheets().values().update(
#         spreadsheetId=spreadsheet_id,
#         range=cell_range,
#         valueInputOption='USER_ENTERED',
#         body=value_range_body
#     )
#     request.execute()

def apply_formula_to_cell_sheet_for_cs(service, spreadsheet_id, cell_range, sheetId):
    formula = f'=HYPERLINK("https://daebongfarmers.notion.site/CS-3f5809dfece649d9986096fce0a1405f", "CS 접수하기(클릭)")'
    value_range_body = {'values': [[formula]]}
    
    request = {
        "updateCells": {
            "range": {
                "sheetId": sheetId,
                "startRowIndex": cell_range["startRowIndex"],
                "endRowIndex": cell_range["endRowIndex"],
                "startColumnIndex": cell_range["startColumnIndex"],
                "endColumnIndex": cell_range["endColumnIndex"]
            },
            "rows": [{
                "values": [{
                    "userEnteredValue": {"formulaValue": formula}
                }]
            }],
            "fields": "userEnteredValue"
        }
    }
    return [request]

In [6]:
def parse_cell_range(cell_address):
    import re

    # 정규식을 사용하여 셀 주소에서 시트 이름, 열, 행을 분리합니다. 시트 이름이 한글을 포함할 수 있도록 수정합니다.
    match = re.match(r"([^!]+)!([A-Z]+)(\d+)", cell_address)
    if not match:
        raise ValueError("Invalid cell address format")

    sheet_name, col, row = match.groups()
    row_index = int(row) - 1  # 1-based index를 0-based index로 변환
    col_index = column_to_index(col) - 1  # 열 이름을 0-based index로 변환

    # 시작 및 끝 인덱스는 하나의 셀이므로 동일합니다.
    return {
        "sheetName": sheet_name,
        "startRowIndex": row_index,
        "endRowIndex": row_index + 1,
        "startColumnIndex": col_index,
        "endColumnIndex": col_index + 1
    }

def column_to_index(column):
    """ 열 이름 (예: 'A', 'B', ..., 'Z', 'AA', etc.)을 0-based index로 변환합니다. """
    index = 0
    for char in column:
        index = index * 26 + (ord(char) - ord('A') + 1)
    return index


In [7]:
def get_sheet_id_by_name(service, spreadsheet_id, sheet_name):
    spreadsheet_paste = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    sheets_paste = spreadsheet_paste.get('sheets', '')
    sheet_order_id = None
    for sheet in sheets_paste:
        if sheet.get('properties', {}).get('title', '') == sheet_name:
            sheet_order_id = sheet.get('properties', {}).get('sheetId', '')
    
    return sheet_order_id

In [8]:
# 색상 코드를 RGB 소수 값으로 변환
def hex_to_rgb_percent(hex):
    hex = hex.lstrip('#')
    return tuple(int(hex[i:i+2], 16)/255.0 for i in (0, 2, 4))


def apply_text(service, spreadsheet_id, cell_range, text):
     # 텍스트를 입력하는 부분
    value_range_body = {
        "values": [[text]]  # 하나의 셀에 텍스트를 입력합니다.
    }
    update_request = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=cell_range,
        valueInputOption='USER_ENTERED',
        body=value_range_body
    )
    update_request.execute()

def apply_styles(service, spreadsheet_id, cell_range, text_size, background_color, bold, sheetId):
    # 셀 스타일을 적용하는 부분
    requests = []
    requests.append({
        "repeatCell": {
            "range": {
                "sheetId": sheetId,  # 첫 번째 시트를 사용, 필요에 따라 수정
                "startRowIndex": cell_range["startRowIndex"],  # API는 0-based index를 사용
                "endRowIndex": cell_range["endRowIndex"],
                "startColumnIndex": cell_range["startColumnIndex"],
                "endColumnIndex": cell_range["endColumnIndex"]
            },
            "cell": {
                "userEnteredFormat": {
                    "backgroundColor": {
                        "red": background_color[0],
                        "green": background_color[1],
                        "blue": background_color[2]
                    },
                    "textFormat": {
                        "fontSize": text_size,
                        "bold": bold
                    }
                }
            },
            "fields": "userEnteredFormat(backgroundColor,textFormat)"
        }
    })

    batch_update_request = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={"requests": requests}
    ).execute()

In [27]:
def apply_dynamic_styles(service, spreadsheet_id, cell_range, text_size, background_color=None, font_color=None, bold=False, sheetId=0):
    # 셀 스타일을 동적으로 적용하는 부분
    requests = []
    cell_format = {
        "textFormat": {
            "fontSize": text_size,
            "bold": bold
        }
    }

    # 배경색 설정이 제공된 경우
    if background_color:
        cell_format["backgroundColor"] = {
            "red": background_color[0],
            "green": background_color[1],
            "blue": background_color[2]
        }

    # 글꼴 색상 설정이 제공된 경우
    if font_color:
        cell_format["textFormat"]["foregroundColor"] = {
            "red": font_color[0],
            "green": font_color[1],
            "blue": font_color[2]
        }

    requests.append({
        "repeatCell": {
            "range": {
                "sheetId": sheetId,
                "startRowIndex": cell_range["startRowIndex"],
                "endRowIndex": cell_range["endRowIndex"],
                "startColumnIndex": cell_range["startColumnIndex"],
                "endColumnIndex": cell_range["endColumnIndex"]
            },
            "cell": {
                "userEnteredFormat": cell_format
            },
            "fields": "userEnteredFormat(backgroundColor,textFormat)"
        }
    })

    # batch_update_request = service.spreadsheets().batchUpdate(
    #     spreadsheetId=spreadsheet_id,
    #     body={"requests": requests}
    # ).execute()

    return requests


In [31]:
def hide_rows(service, spreadsheet_id, sheet_order_id):
    # 숨길 열들의 시작과 끝 인덱스 (I, N, O, T, U, Z, AA, AF, AG, AL, AM)
    rows_to_hide = [0]

    # 각 열에 대해 숨기는 요청 생성
    requests = [{
        "updateDimensionProperties": {
            "range": {
                "sheetId": sheet_order_id,  # 시트 ID
                "dimension": "ROWS",  # 열 변경
                "startIndex": row_index,  # 시작 인덱스
                "endIndex": row_index + 1  # 종료 인덱스 (startIndex와 같은 열을 숨김)
            },
            "properties": {
                "hiddenByUser": True
            },
            "fields": "hiddenByUser"
        }
    } for row_index in rows_to_hide]

    # 요청 실행

    return requests
    response = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={"requests": requests}
    ).execute()

    print(response)

In [38]:
def fixed_view_column(service, spreadsheet_id, sheet_order_id):
    # 고정할 열의 수 설정
    frozen_column_count = 1  # 첫 번째 열만 고정

    # 요청 보내기
    request = {
        "updateSheetProperties": {
            "properties": {
                "sheetId": sheet_order_id,  # 해당 시트 ID
                "gridProperties": {
                    "frozenColumnCount": frozen_column_count  # 고정할 열 수 설정
                }
            },
            "fields": "gridProperties.frozenColumnCount"  # 열 고정 속성만 업데이트
        }
    }
    return [request]

    # response = service.spreadsheets().batchUpdate(
    #     spreadsheetId=spreadsheet_id,
    #     body={"requests": [request]}
    # ).execute()

    print("Fixed first column:", response)

In [40]:
def fixed_view_row(service, spreadsheet_id, sheet_order_id):
    # 고정할 열의 수 설정
    frozen_row_count = 1  # 첫 번째 열만 고정

    # 요청 보내기
    request = {
        "updateSheetProperties": {
            "properties": {
                "sheetId": sheet_order_id,  # 해당 시트 ID
                "gridProperties": {
                     "frozenRowCount": frozen_row_count  # 고정할 행 수 설정
                }
            },
            "fields": "gridProperties.frozenRowCount"  # 행 고정 속성만 업데이트
        }
    }
    return [request]

    # response = service.spreadsheets().batchUpdate(
    #     spreadsheetId=spreadsheet_id,
    #     body={"requests": [request]}
    # ).execute()

    print("Fixed first column:", response)

In [47]:
import time

user_sheet_id = "1Ajz2g-pChLSoies5umhPMcgTp5m2MIjhljJOviPlWCU"

if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    # service_drive = build("drive", "v3", credentials=creds) 
    range_name = "셀러컨택포인트!A2:T112" #2 ~ 112  고정 안 함
    user_data_list = get_values_from_range(service_sheets, user_sheet_id, range_name)
    print(user_data_list)

     
    for user_data in user_data_list:
        time.sleep(1)
        
        if user_data[10]:
            # apply_formula_to_muti_cell_ready(service_sheets, user_data[10], "발주서!N11:N")
            # time.sleep(0.5)
            # apply_formula_to_cell_sheet(service_sheets, user_data[10], "발주서!M8")
            # time.sleep(0.5)

            # apply_text(service_sheets, user_data[10], "발주서!O10", "발주 완료된 건들은 입금 후 출고됩니다. 취소를 원하시면 입금 전 취소 건은 행을 삭제해주세요.")
            # time.sleep(0.5)
            # apply_text(service_sheets, user_data[10], "발주서!N10", "발주상태")
            #FF9902
            #FF9902
            time.sleep(1)
            background_color = hex_to_rgb_percent("FF9902")
            background_color_black = hex_to_rgb_percent("000000")

            range_indices_c6 = parse_cell_range("발주서!C6")
            range_indices_p1 = parse_cell_range("누적발주!P1")
            # apply_styles(service_sheets, user_data[10], range_indices_c6, 13, background_color, True, get_sheet_id_by_name(service_sheets, user_data[10], '발주서'))
            # 발주서_sheet_id = get_sheet_id_by_name(service_sheets, user_data[10], '발주서')
            누적발주_sheet_id = get_sheet_id_by_name(service_sheets, user_data[10], '누적발주')

            # data_c6 = apply_dynamic_styles(service_sheets, user_data[10], range_indices_c6, 15, None, background_color, True, 발주서_sheet_id)
            # time.sleep(0.5)
            # data_p1 = apply_dynamic_styles(service_sheets, user_data[10], range_indices_p1, 11, background_color, background_color_black, True, 누적발주_sheet_id)
            # time.sleep(0.5)

            # data_1 = apply_formula_to_cell_sheet_for_cs(service_sheets, user_data[10], range_indices_c6, 발주서_sheet_id)
            # time.sleep(0.5)
            # data_2 = apply_formula_to_cell_sheet_for_cs(service_sheets, user_data[10], range_indices_p1, 누적발주_sheet_id)

            fixed_data = fixed_view_row(service_sheets, user_data[10], 누적발주_sheet_id)
            # data_c6 + data_p1 + data_1 + data_2 + 

            all_requests = fixed_data

             # 모든 변경 사항을 단일 batchUpdate로 실행
            batch_update_request = service_sheets.spreadsheets().batchUpdate(
                spreadsheetId=user_data[10],
                body={"requests": all_requests}
            ).execute()

        print(user_data[1])

[['', '20', '권오상', '20_권오상', 'O', '802-87-00673', '', '', '', '', '1D86pGcpOsFWp6IWb_KCGnNJerYcIiFNYzptaR2m21WI', 'https://docs.google.com/spreadsheets/d/1D86pGcpOsFWp6IWb_KCGnNJerYcIiFNYzptaR2m21WI', '', '권오상', '', '', '', '', '', '1'], ['', '21', '', '21_', '', '', '', '', '', '', '1U_HI5Th-1Yh9RBWiOGljs-NhXazxPl1HkbPQ9IZrKzA', 'https://docs.google.com/spreadsheets/d/1U_HI5Th-1Yh9RBWiOGljs-NhXazxPl1HkbPQ9IZrKzA', '', '', '', '', '', '', '', '1'], ['', '22', '오늘의과일', '22_오늘의과일', 'O', '4023685107', '', '', '', '', '1RODBFbJ0p49qvpEeMOLLBtllLLUKZkxKX0Dax4V_-ms', 'https://docs.google.com/spreadsheets/d/1RODBFbJ0p49qvpEeMOLLBtllLLUKZkxKX0Dax4V_-ms', '', '', '', '', '', '', '', '1'], ['', '23', '매월상회', '23_매월상회', '오픈채팅', '', '', '', '', '', '1xa1tixBAnCPSuV0YMyTJvBOSqzR9T1M6zeymXGXXbYY', 'https://docs.google.com/spreadsheets/d/1xa1tixBAnCPSuV0YMyTJvBOSqzR9T1M6zeymXGXXbYY', '', '', '', '', '', '', '', '1'], ['', '24', '과일남자', '24_과일남자', 'O', '', 'O', '원영상회', '', '', '1e4kInWbkfgWtuvw7rZ51Bu